In [ ]:
%pip install PyPDF2

In [ ]:
import io
import os
import fitz
import re
from PIL import Image
from google.cloud import vision
import PyPDF2

In [ ]:
# Đường dẫn đến file PDF
file = "raw/Sach-Nom-Cong-Giao-1995-012.pdf"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'nth-wording-440008-v3-5799f3182196.json'
QUOCNGU_CHARSET = 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ'

In [ ]:
def resize_image(image_path, max_size=1000, output_file=None):
    with Image.open(image_path) as img:
        width, height = img.size
        total_size = width + height

        if total_size > max_size:
            scale_factor = max_size / total_size
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            img = img.resize((new_width, new_height), Image.LANCZOS)
            img.save(image_path)
            print(f"Resized image '{image_path}' to {new_width}x{new_height}")

            if output_file:
                output_file.write(f"{os.path.basename(image_path)}\n")
        else:
            print(f"Image '{image_path}' does not need resizing.")
            
def is_quoc_ngu(text):
    return any(char in QUOCNGU_CHARSET for char in text)

In [ ]:
def perform_ocr(image_path, output_file="output.txt"):
    client = vision.ImageAnnotatorClient()
    
    # Mở và đọc hình ảnh từ file
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    # Tạo đối tượng hình ảnh cho Google Vision API
    image = vision.Image(content=content)

    # Gọi Google Vision API để thực hiện OCR
    response = client.document_text_detection(image=image)
    texts = response.text_annotations

    # Ghi kết quả OCR vào file
    with open(output_file, 'a', encoding='utf-8') as f:
        if texts:
            # Lấy văn bản chính duy nhất
            full_text = texts[0].description
            
            # Tách từng dòng và chỉ ghi các dòng có chữ quốc ngữ
            for line in full_text.splitlines():
                if is_quoc_ngu(line):  # Chỉ ghi dòng nếu nó chứa chữ quốc ngữ
                    f.write(line + "\n")
        else:
            f.write("No text detected in the image.\n")

    # Kiểm tra lỗi từ API
    if response.error.message:
        raise Exception(response.error.message)

In [ ]:
def split_pdf(input_pdf, start_page, end_page, output_pdf):
    try:
        # Mở file PDF gốc
        with open(input_pdf, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            writer = PyPDF2.PdfWriter()
            
            # Kiểm tra giá trị hợp lệ
            if start_page < 1 or end_page > len(reader.pages) or start_page > end_page:
                print("Invalid page range.")
                return
            
            # Thêm các trang được chọn vào file mới
            for i in range(start_page - 1, end_page):
                writer.add_page(reader.pages[i])
            
            # Lưu file PDF mới
            with open(output_pdf, 'wb') as output_file:
                writer.write(output_file)
            
            print(f"Đã tách file PDF từ trang {start_page} đến {end_page} và lưu vào '{output_pdf}'.")

    except Exception as e:
        print(f"Lỗi: {e}")

# Ví dụ sử dụng
input_pdf_path = "raw/Sach-Nom-Cong-Giao-1995-012.pdf"  # Thay bằng đường dẫn file gốc
output_pdf_path = "outputPDF.pdf"  # Thay bằng đường dẫn file đầu ra
start_page = int(input("Nhập trang bắt đầu: "))
end_page = int(input("Nhập trang kết thúc: "))
split_pdf(input_pdf_path, start_page, end_page, output_pdf_path)